In [409]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [415]:
class CancerDataset(Dataset):

    def __init__(self, file_name):
        data = pd.read_csv(file_name).iloc[:, 1:-1]
        
        x = data.iloc[:, 1:].values
        y = pd.get_dummies(data.iloc[:, 0]).values
        
        sc = StandardScaler()
        X = sc.fit_transform(x)
        Y = y

        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    
    
full_dataset = CancerDataset('data.csv')
traininig_set, val_train_set = random_split(full_dataset, [455, 114])
validation_set, testing_set = random_split(val_train_set, [57, 57])

training_loader = DataLoader(traininig_set, batch_size=4, shuffle=True)
validation_loader= DataLoader(validation_set, batch_size=8, shuffle=False)
testing_loader = DataLoader(testing_set, batch_size=8, shuffle=False)

In [411]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(30, 2)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(2, 2)
        self.softmax = nn.Softmax(dim=1)

    
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        
        return x

In [412]:
net = Net()

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
criterion = nn.MSELoss()

for epoch in range(4):
    net.train()
    for (inputs, labels) in training_loader:
        optimizer.zero_grad()
        
        output = net(inputs)
        # print(output, labels)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 

In [413]:
PATH = './cancer_net.pth'
torch.save(net.state_dict(), PATH)

In [414]:
correct, total = 0, 0
net.eval()
with torch.no_grad():
    for (inputs, labels) in testing_loader:
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs, 1)
        _, y = torch.max(labels, 1)
        
        total += labels.size(0)
        correct += (predicted == y).sum().item()
        
print(f'Accuracy of nn: {correct / total}')

Accuracy of nn: 0.9473684210526315
